In [1]:
from manage_model import create_NN_model, create_sklearn_model, allowgrowthgpus, load_model
from best_models import retrieve_best_models, retrieve_models_by_id
from check_and_prepare_dataset import load_dataset, prepare_dataset

2023-06-20 17:32:03.722391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 17:32:03.761593: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#filters = ["df_flt['params'].str.contains(\"'dropout': None\")"] # no dropout
filters = ["~df_flt['params'].str.contains(\"'dropout': None\")"] # with dropout
opt_targets = ['auc', 'f1-score 1', 'hybrid1', 'hybrid2', 'hybrid5', 'NH2', 'NH5', 'NH10']
resdir = '/mnt/nvme2tb/ffp/results/bestmodels'
testfpattern=cvrespattern = '*NN_ns*mean*'
testmodel = retrieve_models_by_id(197,'auc test', resdir, testfpattern, opt_targets, 'val.', 'test', filters, 3)
testmodels = retrieve_best_models(resdir, testfpattern, opt_targets, 'val.', 'test', filters, 3)



In [3]:
testmodel

{'params': {'ES_mindelta': 0.002,
  'ES_monitor': 'val_loss',
  'ES_patience': 10,
  'batch_size': 512,
  'class_weights': {0: 1, 1: 2},
  'dropout': 0.3,
  'feature_drop': ('month',
   'weekday',
   'dom_dir',
   'dir_max',
   'pop',
   'corine_gr1',
   'corine_gr2',
   'corine_gr3',
   'corine_gr4',
   'corine_gr5',
   'corine_gr6',
   'corine_gr7',
   'corine_gr8',
   'corine_gr9'),
  'max_epochs': 2000,
  'metric': 'accuracy',
  'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}),
  'optimizer': {'adam_params': None, 'name': 'Adam'}},
 'trial': 197}

In [4]:
X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         testmodel['params']['feature_drop'])

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_8', 'bin_corine_gr33', 'bin_weekday_6', 'bin_corine_gr21', 'bin_corine_gr24', 'bin_dom_dir_2', 'bin_dom_dir_4', 'bin_month_7', 'bin_dir_max_3', 'bin_dom_dir_1', 'bin_month_10', 'bin_dom_dir_8', 'bin_dir_max_1', 'bin_dom_dir_3', 'bin_dir_max_4', 'bin_weekday_2', 'bin_weekday_1', 'bin_corine_gr32', 'bin_dom_dir_7', 'bin_month_9', 'bin_corine_gr4', 'pop', 'bin_month_8', 'bin_corine_gr1', 'bin_weekday_5', 'bin_month_3', 'bin_dom_dir_5', 'bin_weekday_4', 'bin_corine_gr22', 'bin_month_4', 'bin_corine_gr31', 'bin_dir_max_5', 'bin_weekday_7', 'bin_dir_max_2', 'bin_dom_dir_6', 'bin_corine_gr23', 'bin_corine_gr5', 'bin_month_5', 'bin_dir_max_7', 'bin_dir_max_6', 'bin_week

In [5]:
allowgrowthgpus()

2023-06-20 17:34:39.354520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-20 17:34:39.369843: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-20 17:34:39.369986: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-20 17:34:39.371293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other 

1 Physical GPUs, 1 Logical GPUs


In [7]:
model=load_model('/mnt/nvme2tb/ffp/results/bestmodels/models/hypres_tf_ns_ncv_do_2019_weights_id_197_r_0_auctest_1.cpkt', 'tf',\
                 testmodel['params'],X)

In [8]:
model.predict(X)

 319/1303 [======>.......................] - ETA: 0s

2023-06-20 17:35:14.044429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1303/1303 [==============================] - 2s 459us/step


array([[0.23376964, 0.76623034],
       [0.51400113, 0.48599887],
       [0.01781548, 0.9821845 ],
       ...,
       [0.01311231, 0.98688775],
       [0.03468338, 0.9653166 ],
       [0.04352969, 0.9564703 ]], dtype=float32)